In [ ]:
# !pip install tensorflow_text
import tensorflow_text

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow import string
from keras.models import Model
from keras.layers import Input, Dense

In [ ]:
df = pd.read_csv("/content/sms_spam.csv")
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


#### Bert Model

In [ ]:
preprocessor_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"
encoder_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-4-h-512-a-8/1"

bert_preprocessor = hub.KerasLayer(preprocessor_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [ ]:
# DNN by the help of Functional API
def neural_network_model(bert_preprocessor, bert_model):
  input = Input(shape=(), dtype="string")
  encoder_input = bert_preprocessor(input)
  encoder_output = bert_model(encoder_input)
  pooled_output = encoder_output['pooled_output']
  h1 = Dense(10, activation='relu')(pooled_output)
  output = Dense(1, activation='sigmoid')(h1)
  bert_model = Model(inputs=input, outputs=output)
  bert_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return bert_model

In [ ]:
bert_model = neural_network_model(bert_preprocessor, bert_encoder)
bert_model.fit(df['text'], df['label'], epochs=2, batch_size=64)

Epoch 1/2
88/88 [==============================] - 54s 601ms/step - loss: 0.1517 - accuracy: 0.9447
Epoch 2/2
88/88 [==============================] - 53s 601ms/step - loss: 0.0764 - accuracy: 0.9786


#### Validation

In [ ]:
df['predictions'] = bert_model.predict(df['text'])

df['predictions'] = df['predictions'].apply(lambda x: "spam" if x>0.5 else "not spam")

175/175 [==============================] - 49s 277ms/step


In [ ]:
df['label'].value_counts()

,count
label,
0,4825
1,747


In [ ]:
df['predictions'].value_counts()

,count
predictions,
not spam,4872
spam,700


In [ ]:
x = bert_model.predict(np.array(["Hello, how are you?"]))
print(x)
if x<0.5:
  print("not spam")
else:
  print("spam")

1/1 [==============================] - 0s 60ms/step
[[0.00315927]]
not spam
